In [50]:
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine
load_dotenv()
from utils import generate_conn_string

conn_str = generate_conn_string('projects')

In [55]:
df = pd.read_sql_table('otomoto_raw', con=conn_str)

In [56]:
from pandas import DataFrame

def clean_df(df: DataFrame, y_col: str, fillna_dict: dict) -> DataFrame:
    df = df.drop_duplicates()
    df = df.dropna(how="all")
    df = df.drop(columns=['index', 'generacja','wersja'], errors='ignore')

    df.reset_index(inplace=True, drop=True)

    idxs_to_drop = []
    for i in range(0,len(df)):
        row = df.iloc[i]
        cols_with_nans = []
        for col in df.columns:
            if not row[col]:
                cols_with_nans.append(col)

        if len(cols_with_nans)/len(df.columns) > 0.45 or not row[y_col]:
            idxs_to_drop.append(i)

    df = df.drop(idxs_to_drop)

    for key,val in fillna_dict.items():
        df[key] = df[key].fillna(val)

    return df


d = {'serwisowany_w_aso':'Nie',
     'bezwypadkowy':'Nie',
     'stan':'Używany'}

df = clean_df(df, y_col='cena', fillna_dict=d)





In [65]:
def convert_to(s: str, dtype: int|float):

    if isinstance(s, str):
        s = s.lower().replace("km","")
        s = s.lower().replace('l/100km','')
        s = s.lower().replace('l/100','')

        s = s.lower().replace('cm3','')
        s = s.replace(" ","")
        s = s.replace(',','.')
        s = s.strip()
    
    if not s:
        return dtype(0)

    return dtype(s)

for col in ['cena','przebieg','pojemosc_skokowa','spalanie_w_miescie']:
    df[col]= df[col].apply(convert_to, dtype=float)

for col in ['rok_produkcji','moc','liczba_drzwi','liczba_miejsc']:
    df[col]= df[col].apply(convert_to, dtype=int)

df['stan'] = df['stan'].apply(lambda x: x.replace('Używane','Używany').replace('Nowe','Nowy'))


In [69]:
from sklearn.model_selection import train_test_split

y = df[['cena']]
X = df[[col for col in df.columns if col != 'cena']]

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [75]:

df.to_sql('preprocessed',con=conn_str, schema='otomoto')

X_test.to_sql('x_test',con=conn_str, schema='otomoto')
y_test.to_sql('y_test',con=conn_str, schema='otomoto')

X_train.to_sql('x_train',con=conn_str, schema='otomoto')
y_train.to_sql('y_train',con=conn_str, schema='otomoto')

685